In [ ]:
import ipywidgets as widgets
import pandas as pd
from IPython.display import display
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import logging

# File paths for the different tables
file_paths = {
    "Table1 - Fused Silica Nonlinearity": "INPUT/Table1.xlsx",
    "Table2A - Second-order NLO properties of bulk materials": "INPUT/Table2A.xlsx",
    "Table2B - Third-order NLO properties of bulk materials": "INPUT/Table2B.xlsx",
    "Table3 - NLO measurement results for solvents": "INPUT/Table3.xlsx",
    "Table4A - Second-order NLO properties of 0D–1D–2D materials": "INPUT/Table4A.xlsx",
    "Table4B - Third-order NLO properties of 0D–1D–2D materials": "INPUT/Table4B.xlsx",
    "Table5A - Second-order NLO properties of metamaterials": "INPUT/Table5A.xlsx",
    "Table5B - Third-order NLO properties of metamaterials": "INPUT/Table5B.xlsx",
    "Table6 - NLO properties of fiber waveguiding materials": "INPUT/Table6.xlsx",
    "Table7A - Second-order NLO properties of on-chip waveguiding materials": "INPUT/Table7A.xlsx",
    "Table7B - Third-order NLO properties of on-chip waveguiding materials": "INPUT/Table7B.xlsx",
    "Table8A - NLO properties of hybrid fibers": "INPUT/Table8A.xlsx",
    "Table8B - NLO properties of hybrid on-chip waveguides": "INPUT/Table8B.xlsx"
}

logging.basicConfig(level=logging.INFO)

# Widgets
loading_label = widgets.Label(value="Loading...", layout=widgets.Layout(display='none'))
dropdown = widgets.Dropdown(options=file_paths.keys(), description='Select Table:')
search_column_dropdown = widgets.Dropdown(description='Search Column:')
search_input = widgets.Text(description='Search Text:')
search_button = widgets.Button(description='Search', button_style='info')
output = widgets.Output()
table = widgets.GridspecLayout(1, 1, grid_gap="5px")
checked_values_label = widgets.Label(value="Checked Values:")  
scatter_plot_output = widgets.Output()  

# Global variable to store the DataFrame
current_df = pd.DataFrame()
global max_column_widths
checkboxes = [] 

def display_scrollable_table(filtered_data):
    global table

    num_rows, num_cols = filtered_data.shape
    output.clear_output()
    
    header_widgets = [widgets.Label(value=header, layout=widgets.Layout(width="150px")) for header in filtered_data.columns]
    headers_box = widgets.HBox(header_widgets)

    data_widgets = []
    for i in range(num_rows):
        row_widgets = []
        for j in range(num_cols):
            display_value = '' if pd.isna(filtered_data.iloc[i, j]) else str(filtered_data.iloc[i, j])
            row_widgets.append(widgets.Label(value=display_value, layout=widgets.Layout(width="150px")))
        data_widgets.append(widgets.HBox(row_widgets))

def get_max_column_width(data, headers):
    max_lengths = [len(header) for header in headers]
    for row in data.itertuples(index=False):
        for i, cell in enumerate(row):
            if i < len(max_lengths):
                max_lengths[i] = max(max_lengths[i], len(str(cell)))
    return max_lengths

def plot_scatter(selected_column):
    global current_df
    checked_values = []

    # Collect values from checked checkboxes
    for i, checkbox in enumerate(checkboxes):
        if checkbox.value:  
            value = current_df[selected_column].iloc[i]
            try:
                # Attempt to convert to a float
                numeric_value = pd.to_numeric(value, errors='raise')  # Raise an error for invalid values
                checked_values.append(numeric_value)
            except ValueError:
                pass 

    # Plot if there are valid checked values
    if checked_values:
        plt.figure(figsize=(10, 5))
        plt.scatter(range(len(checked_values)), checked_values, label=selected_column)
        plt.title(f"Scatter Plot of {selected_column} for Checked Rows")
        plt.xlabel("Row Index")
        plt.ylabel(selected_column)
        plt.axhline(0, color='gray', linewidth=0.5, linestyle='--')
        plt.grid()
        plt.legend()
        plt.show()
    else:
        print("No valid numeric values selected for plotting.")

def display_table(filtered_data):
    global table
    num_rows, num_cols = filtered_data.shape

    with output:
        output.clear_output()
        display(loading_label)
        loading_label.layout.display = 'block'

    # Clear the previous table and reinitialize it with the correct size
    table = widgets.GridspecLayout(num_rows + 1, num_cols + 1, grid_gap="5px",
                                   layout=widgets.Layout(align_items="center"))

    # Checkbox Header
    table[0, 0] = widgets.Label(value="Select Row", layout=widgets.Layout(width="100px"))

    # Headers: Add clickable buttons for the headers
    for i, header in enumerate(filtered_data.columns):
        button = widgets.Button(description=header, layout=widgets.Layout(width=f"{max_column_widths[i]*10}px"))
        button.on_click(lambda b, col=header: handle_column_click(col))
        table[0, i + 1] = button

    # Data: Add checkboxes for each row and labels for each cell
    for i in range(num_rows):
        checkbox = widgets.Checkbox(
            value=False,
            layout=widgets.Layout(width="auto", align_self="center", justify_self="center"))
        checkboxes.append(checkbox)
        checkbox.observe(lambda change, index=i: update_plot_on_checkbox_change(index), names='value')
        table[i + 1, 0] = checkbox
        for j, header in enumerate(filtered_data.columns):
            value = filtered_data.iloc[i, j]
            display_value = '' if pd.isna(value) else str(value)
            table[i + 1, j + 1] = widgets.Label(value=display_value, layout=widgets.Layout(width=f"{max_column_widths[j]*10}px"))

    # Display the table
    loading_label.layout.display = 'none'
    with output:
        output.clear_output()
        display(checked_values_label)
        display(scatter_plot_output) 
        display(table)

# Event handler for file selection from dropdown
def on_table_select(change):
    global current_df
    global max_column_widths
    
    with output:
        output.clear_output()
        display(loading_label)
        loading_label.layout.display = 'block'
        
    with output:
        output.clear_output()

        # Load the selected table
        selected_table = dropdown.value
        file_path = file_paths[selected_table]
        current_df = pd.read_excel(file_path)

        # Update search column dropdown based on loaded columns
        search_column_dropdown.options = current_df.columns
        headers = current_df.columns.tolist()
        max_column_widths = get_max_column_width(current_df, headers)

        display_table(current_df)

def update_plot_on_checkbox_change(change):
    print("Checkbox changed:", change)

    
# Event handler for search button
def on_search_button_click(b):
    search_button.disabled = True 
    
    with output:
        output.clear_output()
        display(loading_label)
        loading_label.layout.display = 'block'
        
    global max_column_widths
    global current_df
    global first_search_call
    search_col = search_column_dropdown.value
    search_text = search_input.value

    if search_col and search_text:
        filtered_data = current_df[current_df[search_col].astype(str).str.contains(search_text, case=False, na=False)]
        if first_search_call:
            headers = current_df.columns.tolist()
            max_column_widths = get_max_column_width(current_df, headers)
            first_search_call = False
        display_table(filtered_data)
    else:
        if first_search_call:
            headers = current_df.columns.tolist()
            max_column_widths = get_max_column_width(current_df, headers)
            first_search_call = False
        display_table(current_df)
    loading_label.layout.display = 'none'
    search_button.disabled = False
    
def handle_column_click(column_name):
    global current_df
    checked_values = []

    # Collect values from checked checkboxes
    for i, checkbox in enumerate(checkboxes):
        if checkbox.value:  # If the checkbox is checked
            checked_values.append(current_df[column_name].iloc[i])

    # Update the checked values label
    checked_values_label.value = "Checked Values: " + ', '.join(map(str, checked_values)) if checked_values else "Checked Values: None"
    
    # Generate and display the scatter plot
    scatter_plot_output.clear_output()
    with scatter_plot_output:
        plot_scatter(column_name)

# Attach event handlers
dropdown.observe(on_table_select, names='value')
search_button.on_click(on_search_button_click)

# Display UI
display(dropdown, search_column_dropdown, search_input, search_button, output)
